In [1]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.core.display import display, HTML

def scrape_table(table):
    df = pd.read_html(str(table))
    df = pd.DataFrame(df[0])
    return Table.from_df(df)

C:\Users\Anees' Laptop\AppData\Local\Temp\ipykernel_4696\1145054876.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


#1. Scrape data#

In [3]:
egg_price_url = "https://www.globalproductprices.com/rankings/egg_prices/"
tax_rate_url = "https://tradingeconomics.com/country-list/sales-tax-rate"
currency_url = "https://taxsummaries.pwc.com/glossary/currency-codes"

In [4]:
#Scrape data from egg_price_url
egg_price_request = requests.get(egg_price_url)
egg_soup = BeautifulSoup(egg_price_request.text, 'html.parser')

raw_price_table = egg_soup.find('table')
egg_price_table = scrape_table(raw_price_table)
egg_price_table = egg_price_table.sort("Countries")
egg_price_table

Countries,"Eggs prices, dozen",Rank
Argentina,3.48,25
Australia,5.08,5
Austria,3.81,17
Azerbaijan,1.59,76
Bangladesh,1.25,79
Belgium,4.13,10
Bolivia,1.87,70
Brazil,2.18,57
Bulgaria,4.87,6
Cameroon,1.93,68


In [5]:
#Scrape data from tax_rate_url
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
tax_rate_request = requests.get(tax_rate_url, headers=headers)
tax_soup = BeautifulSoup(tax_rate_request.text, 'html.parser')

raw_tax_table = tax_soup.find('table')
tax_price_table = scrape_table(raw_tax_table)
tax_price_table

Country,Last,Previous,Reference,Unit
Afghanistan,10,10,Dec/24,%
Albania,20,20,Dec/24,%
Algeria,19,19,Dec/24,%
Andorra,4.5,4.5,Dec/24,%
Angola,14,14,Dec/23,%
Antigua And Barbuda,15,15,Dec/24,%
Argentina,21,21,Dec/24,%
Armenia,20,20,Dec/24,%
Aruba,4,4,Dec/24,%
Australia,10,10,Dec/24,%


In [6]:
#Collect currency data
currency_request = requests.get(currency_url, headers=headers)
currency_soup = BeautifulSoup(currency_request.text, 'html.parser')

raw_currency_table = currency_soup.find('table')
currency_df = pd.read_html(str(raw_currency_table), header=0)[0]
currency_table = Table.from_df(currency_df)
currency_table

Territory,Currency code (ISO 4217),Currency name
Albania,ALL,lek
Algeria,DZD,dinar
Angola,AOA,kwanza
Argentina,ARS,peso
Armenia,AMD,dram
Australia,AUD,dollar
Austria,EUR,euro
Azerbaijan,AZN,manat
Bahrain,BHD,dinar
Barbados,BBD,dollar


2. Data Cleaning

In [ ]:
missing_country_mapping = {
    "United States" : "USA",
    "United Kingdom" : "UK",
    "Dominican Republic" : 'Domin. Rep.',
    "United Arab Emirates": "UA Emirates",
    "Czech Republic" : "Czechia"

}

tax_price_table = tax_price_table.with_columns("Country",[missing_country_mapping.get(x, x) for x in tax_price_table.column("Country")])
tax_price_table

Country,Last,Previous,Reference,Unit
Afghanistan,10,10,Dec/24,%
Albania,20,20,Dec/24,%
Algeria,19,19,Dec/24,%
Andorra,4.5,4.5,Dec/24,%
Angola,14,14,Dec/23,%
Antigua And Barbuda,15,15,Dec/24,%
Argentina,21,21,Dec/24,%
Armenia,20,20,Dec/24,%
Aruba,4,4,Dec/24,%
Australia,10,10,Dec/24,%


In [19]:
from rapidfuzz import process, fuzz

def create_fuzzy_mapping(source, target):
    mapping = {}
    low_score_matches = []
    for country in source:
        match = process.extractOne(country, target, scorer=fuzz.token_sort_ratio)
        if match and match[1] > 30: 
            mapping[country] = match[0]
        elif 50 < match[1] <= 80:  # Low-confidence match
            low_score_matches.append((country, match[0], match[1]))
    return mapping
    

fuzzy_mapping = create_fuzzy_mapping(egg_price_table.column('Countries'), tax_price_table.column('Country'))

# Function to standardize using fuzzy mapping
def standardize_country(name):
    return fuzzy_mapping.get(name, name)


# Add standardized columns to each table
egg_price_table = egg_price_table.with_column('Standardized_Country', egg_price_table.apply(standardize_country, 'Countries'))
tax_price_table = tax_price_table.with_column('Standardized_Country', tax_price_table.apply(standardize_country, 'Country'))

# Find common standardized countries
common_countries = set(egg_price_table.column('Standardized_Country')).intersection(tax_price_table.column('Standardized_Country'))

# Filter rows with common countries
filtered_table1 = egg_price_table.where('Standardized_Country', lambda x: x in common_countries)
filtered_table2 = tax_price_table.where('Standardized_Country', lambda x: x in common_countries)

# Display the filtered tables
filtered_table1.show()
filtered_table2.show()


Countries,"Eggs prices, dozen",Rank,Standardized_Country
Argentina,3.48,25,Argentina
Australia,5.08,5,Australia
Austria,3.81,17,Austria
Azerbaijan,1.59,76,Azerbaijan
Bangladesh,1.25,79,Bangladesh
Belgium,4.13,10,Belgium
Bolivia,1.87,70,Bolivia
Brazil,2.18,57,Brazil
Bulgaria,4.87,6,Bulgaria
Cameroon,1.93,68,Cameroon


Country,Last,Previous,Reference,Unit,Standardized_Country
Argentina,21,21,Dec/24,%,Argentina
Australia,10,10,Dec/24,%,Australia
Austria,20,20,Dec/24,%,Austria
Azerbaijan,18,18,Dec/24,%,Azerbaijan
Bangladesh,15,15,Dec/24,%,Bangladesh
Belgium,21,21,Dec/24,%,Belgium
Bolivia,13,13,Dec/24,%,Bolivia
Brazil,17,17,Dec/24,%,Brazil
Bulgaria,20,20,Dec/24,%,Bulgaria
Cameroon,19.25,19.25,Dec/24,%,Cameroon


In [13]:
low_score_matches

NameError: name 'low_score_matches' is not defined

In [134]:
egg_countries_set = set(egg_price_table.column("Countries"))
tax_rate_countries_set = set(final_tax_price_table.column("Country"))

missing_countries1 = egg_countries_set - tax_rate_countries_set

print(f"Missing countries are: {missing_countries1}")

Missing countries are: {'Kuwait', 'Puerto Rico', 'Hong Kong'}


Country,Sales Tax Rate (%)
Argentina,21
Australia,10
Austria,20
Azerbaijan,18
Bangladesh,15
Belgium,21
Bolivia,13
Brazil,17
Bulgaria,20
Cameroon,19.25


In [152]:
final_currency_table = currency_table.where("Territory", lambda c: c in egg_price_table.column('Countries'))
#final_currency_table = final_currency_table.drop(["Number", "Currency"])
final_currency_table

Territory,Currency code (ISO 4217),Currency name
Argentina,ARS,peso
Australia,AUD,dollar
Austria,EUR,euro
Azerbaijan,AZN,manat
Belgium,EUR,euro
Bolivia,BOB,boliviano
Brazil,BRL,real
Bulgaria,BGN,lev
Canada,CAD,dollar
Chile,CLP,peso


In [127]:
currency_countries_set = set(final_currency_table.column("Country"))

missing_countries2 = egg_countries_set - currency_countries_set

print(f"Missing countries are: {missing_countries2}")

Missing countries are: {'Sri Lanka', 'Hong Kong', 'Vietnam', 'Ivory Coast', 'USA', 'South Africa', 'UA Emirates', 'Tanzania', 'Costa Rica', 'Bolivia', 'Russia', 'New Zealand', 'UK', 'South Korea', 'Saudi Arabia', 'Netherlands', 'Domin. Rep.', 'Puerto Rico', 'Philippines', 'Czechia'}


In [122]:
data = egg_price_table.with_columns("Sales Tax Rate (%)", final_tax_price_table.column("Sales Tax Rate (%)"))
data

ValueError: Column length mismatch. New column does not have the same number of rows as table.